# Semana 7 do Aceleradev DS Codenation

### Professor: Túlio Vieira de Souza | Tema: Feature Engineering

In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Dataset: House Pricing from [Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [46]:
columns = df.columns
types = df.dtypes
missing_ratio = df.isna().sum()/df.shape[0]
unique = df.nunique()
overview = pd.DataFrame({'column': columns,
                        'type': types,
                        'missing_ratio': missing_ratio,
                        'n_unique': unique})
overview.reset_index(inplace=True)
overview.drop(columns='index', inplace=True)
overview

,column,type,missing_ratio,n_unique
0,Id,int64,0.000000,1460
1,MSSubClass,int64,0.000000,15
2,MSZoning,object,0.000000,5
3,LotFrontage,float64,0.177397,110
4,LotArea,int64,0.000000,1073
...,...,...,...,...
76,MoSold,int64,0.000000,12
77,YrSold,int64,0.000000,5
78,SaleType,object,0.000000,9
79,SaleCondition,object,0.000000,6


Vamos analisar o número de variáveis com até 20% de valores faltantes

In [156]:
print(f'número de colunas com até 20% dos dados faltantes: {overview[overview["missing_ratio"] <= .2].shape[0]}\n'
f'número de colunas com mais de 20% dos dados faltantes: {overview[overview["missing_ratio"] > .2].shape[0]}')

número de colunas com até 20% dos dados faltantes: 76
número de colunas com mais de 20% dos dados faltantes: 5


In [78]:
overview['complete'] = list('y' if x == 0 else 'n' for x in overview['missing_ratio'])
overview.head()

,column,type,missing_ratio,n_unique,complete
0,Id,int64,0.000000,1460,y
1,MSSubClass,int64,0.000000,15,y
2,MSZoning,object,0.000000,5,y
3,LotFrontage,float64,0.177397,110,n
4,LotArea,int64,0.000000,1073,y


Vamos obter as colunas de tipo **categórico** e as de tipo **numérico**

In [157]:
columns_complete_num = overview.query('complete == "y" and type != "object"')['column']

In [158]:
columns_complete_cat = overview[(overview['type'] == 'object') & (overview['complete'] == 'y')]['column']

### Variáveis Categóricas

**Label Encoding**

Nessa parte, vamos categorizar variáveis com a biblioteca **sklearn**

In [162]:
columns_complete_cat

2          MSZoning
5            Street
7          LotShape
8       LandContour
9         Utilities
10        LotConfig
11        LandSlope
12     Neighborhood
13       Condition1
14       Condition2
15         BldgType
16       HouseStyle
21        RoofStyle
22         RoofMatl
23      Exterior1st
24      Exterior2nd
27        ExterQual
28        ExterCond
29       Foundation
39          Heating
40        HeatingQC
41       CentralAir
53      KitchenQual
55       Functional
65       PavedDrive
78         SaleType
79    SaleCondition
Name: column, dtype: object

In [163]:
columns_complete_num

0                Id
1        MSSubClass
4           LotArea
17      OverallQual
18      OverallCond
19        YearBuilt
20     YearRemodAdd
34       BsmtFinSF1
36       BsmtFinSF2
37        BsmtUnfSF
38      TotalBsmtSF
43         1stFlrSF
44         2ndFlrSF
45     LowQualFinSF
46        GrLivArea
47     BsmtFullBath
48     BsmtHalfBath
49         FullBath
50         HalfBath
51     BedroomAbvGr
52     KitchenAbvGr
54     TotRmsAbvGrd
56       Fireplaces
61       GarageCars
62       GarageArea
66       WoodDeckSF
67      OpenPorchSF
68    EnclosedPorch
69        3SsnPorch
70      ScreenPorch
71         PoolArea
75          MiscVal
76           MoSold
77           YrSold
80        SalePrice
Name: column, dtype: object

**label encoder** do sklearn retorna categorias **numéricas**

In [102]:
from sklearn.preprocessing import LabelEncoder

In [104]:
encod = LabelEncoder()

Vamos escolher uma variável numérica para vermos como o LabelEncoder funciona

Variável numérica `TotRmsAbvGrd`

In [140]:
var_num = encod.fit_transform(df['TotRmsAbvGrd'])

In [141]:
pd.DataFrame({'original': df['TotRmsAbvGrd'],
             'fit_transform': var_num})

,original,fit_transform
0,8,6
1,6,4
2,6,4
3,7,5
4,9,7
...,...,...
1455,7,5
1456,7,5
1457,9,7
1458,5,3


Como podemos perceber, cada valor numérico original recebeu uma categoria numérica.

Vamos usar uma coluna já categórica agora

Variável categórica `MSZoning`

In [153]:
var_cat = encod.fit_transform(df['MSZoning'])
var_cat

array([3, 3, 3, ..., 3, 3, 3])

In [155]:
pd.DataFrame({'original': df['MSZoning'],
             'fit_transform': var_cat})

,original,fit_transform
0,RL,3
1,RL,3
2,RL,3
3,RL,3
4,RL,3
...,...,...
1455,RL,3
1456,RL,3
1457,RL,3
1458,RL,3


Podemos ver nesse *dataframe* que o LabelEncoder transformou o original em categorias numéricas. Mas como um modelo enxerga tais numeros? será que ele não atribui pesos diferentes às categorias? pois 3 é maior que 1, mas 7 é maior que 3. Em que momento fazer uma categorização numérica dessas não influencia no treinamento de um modelo? Pois, dessa maneira, é perfeitamente possível interpretar esses números categóricos como números mesmo

### Variáveis Numéricas

**Min Max Scaler**

Como já vimos, isso nada mais é que uma **Normalizaçao**, onde os valores ficarão confinados num intervalo $[0,1]$. Se existirem valores negativos na *feature* numérica, esse intervalo será de $[-1,1]$. E para obter o valor normalizado

$$scaled_i=\frac{x_i - x_{min}}{x_{max} - x_{min}}$$

Tanto os `np.array()` quanto `pd.Series()` facilitam muito a normalização de uma lista numérica

Variável numérica `TotRmsAbvGrd`

In [164]:
var_norm = (df['TotRmsAbvGrd'] - df['TotRmsAbvGrd'].min()) / (df['TotRmsAbvGrd'].max() - df['TotRmsAbvGrd'].min())

In [170]:
pd.DataFrame({'original': df['TotRmsAbvGrd'], 'scaled': var_norm})

,original,scaled
0,8,0.500000
1,6,0.333333
2,6,0.333333
3,7,0.416667
4,9,0.583333
...,...,...
1455,7,0.416667
1456,7,0.416667
1457,9,0.583333
1458,5,0.250000


O pacote **sklearn** possui uma função de normalização também

In [212]:
from sklearn.preprocessing import MinMaxScaler

In [226]:
scale_zero_one = MinMaxScaler(feature_range=(0, 1))
scale_zero_two = MinMaxScaler(feature_range=(0, 2))

In [227]:
scaled_zero_one = scale_zero_one.fit_transform(df[['TotRmsAbvGrd']])
scaled_zero_two = scale_zero_two.fit_transform(df[['TotRmsAbvGrd']])

In [228]:
pd.DataFrame({'original': df['TotRmsAbvGrd'], 'scaled': var_norm,
              '[0, 1]': scaled_zero_one[:, 0], 
              '[0, 2]': scaled_zero_two[:, 0]})

,original,scaled,"[0, 1]","[0, 2]"
0,8,0.500000,0.500000,1.000000
1,6,0.333333,0.333333,0.666667
2,6,0.333333,0.333333,0.666667
3,7,0.416667,0.416667,0.833333
4,9,0.583333,0.583333,1.166667
...,...,...,...,...
1455,7,0.416667,0.416667,0.833333
1456,7,0.416667,0.416667,0.833333
1457,9,0.583333,0.583333,1.166667
1458,5,0.250000,0.250000,0.500000


**Standard Scaler**

Também já vista, a **padronização** gera um conjunto de $\overline{x}=0$ e $s=1$, sendo que os valores padronizados podem variar de $]-\infty,\infty[$, onde somente os *outliers* se tornam valores extremos. A idéia da padronização é entender **quantos desvio-padrão** um determinador valor apresenta. E para obtermos esse conjunto

$$z_i=\frac{x_i-\overline{x}}{s}$$

Vamos usar a variável `TotRmsAbvGrd`

In [229]:
stand = (df['TotRmsAbvGrd'] - df['TotRmsAbvGrd'].mean()) / df['TotRmsAbvGrd'].std()

In [231]:
pd.DataFrame({'original': df['TotRmsAbvGrd'], 'standardized': stand})

,original,standardized
0,8,0.911897
1,6,-0.318574
2,6,-0.318574
3,7,0.296662
4,9,1.527133
...,...,...
1455,7,0.296662
1456,7,0.296662
1457,9,1.527133
1458,5,-0.933810


Podemos gerar esses mesmos valores com o **sklearn**

In [232]:
from sklearn.preprocessing import StandardScaler

In [234]:
std_scal = StandardScaler()

In [237]:
std_scaled = std_scal.fit_transform(df[['TotRmsAbvGrd']])

In [238]:
pd.DataFrame({'original': df['TotRmsAbvGrd'], 'standardized': stand, 'standardized_sklearn': std_scaled[:, 0]})

,original,standardized,standardized_sklearn
0,8,0.911897,0.912210
1,6,-0.318574,-0.318683
2,6,-0.318574,-0.318683
3,7,0.296662,0.296763
4,9,1.527133,1.527656
...,...,...,...
1455,7,0.296662,0.296763
1456,7,0.296662,0.296763
1457,9,1.527133,1.527656
1458,5,-0.933810,-0.934130


### Binarização

Esse processo categotiza, 0 ou 1, variáveis numéricas tendo um limite como parâmetro. 

Vamos usar a variável `TotRmsAbvGrd`

In [250]:
binary = df['TotRmsAbvGrd'] >= 5
binary  # dessa maneira não temos 0 ou 1 própriamente ditos

0       True
1       True
2       True
3       True
4       True
        ... 
1455    True
1456    True
1457    True
1458    True
1459    True
Name: TotRmsAbvGrd, Length: 1460, dtype: bool

O **sklearn** possui uma implementação de binarização

In [309]:
from sklearn.preprocessing import Binarizer

In [310]:
to_binarize = Binarizer(threshold=4.9)

In [312]:
binary_skl = to_binarize.fit_transform(df[['TotRmsAbvGrd']])

In [322]:
pd.DataFrame({'original': df['TotRmsAbvGrd'],
             'binary': binary,
             'binary_skl': binary_skl.flatten()})

,original,binary,binary_skl
0,8,True,1
1,6,True,1
2,6,True,1
3,7,True,1
4,9,True,1
...,...,...,...
1455,7,True,1
1456,7,True,1
1457,9,True,1
1458,5,True,1


### Max Absolute Scaler

In [315]:
from sklearn.preprocessing import MaxAbsScaler

In [317]:
max_abs_scaler = MaxAbsScaler()

In [321]:
max_abs_scaled = max_abs_scaler.fit_transform(df[['TotRmsAbvGrd']])
max_abs_scaled.flatten()

array([0.57142857, 0.42857143, 0.42857143, ..., 0.64285714, 0.35714286,
       0.42857143])

In [325]:
pd.DataFrame({'original': df['TotRmsAbvGrd'],
             'normalized_0_1': var_norm,
              'max_abs_scaled': max_abs_scaled.flatten(),
              'standardized_sklearn': std_scaled.flatten(),
             'binary_skl': binary_skl.flatten()})

,original,normalized_0_1,max_abs_scaled,standardized_sklearn,binary_skl
0,8,0.500000,0.571429,0.912210,1
1,6,0.333333,0.428571,-0.318683,1
2,6,0.333333,0.428571,-0.318683,1
3,7,0.416667,0.500000,0.296763,1
4,9,0.583333,0.642857,1.527656,1
...,...,...,...,...,...
1455,7,0.416667,0.500000,0.296763,1
1456,7,0.416667,0.500000,0.296763,1
1457,9,0.583333,0.642857,1.527656,1
1458,5,0.250000,0.357143,-0.934130,1
